In [ ]:
import numpy as np
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import h5py
import os

import keras
import keras.layers as KL
import keras.models as KM
from keras.callbacks import ModelCheckpoint

import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

from data_utils.hdf5_sequence import HDF5Sequence
from get_model import get_model
from constants import MODEL_NAME, MODEL_SAVE_PATH, CHECKPOINT_PATH
import utils

%load_ext autoreload
%autoreload 2

## Constants

In [ ]:
PRETRAINED_MODEL_NAME = 'hierdec-trio_16bar'
config = configs.CONFIG_MAP[PRETRAINED_MODEL_NAME]

DATA_PATH = './data/lmd_full/'
HDF5_PATH = DATA_PATH + 'lmd_full_split.h5'

# Definitions moved to get_model
# TIMESTEPS = 256
# DIM_MELODY = 90
# DIM_LATENT = 512

In [ ]:
config.hparams

## Hyperparameters

In [ ]:
batch_size = 16

## Data pipeline

In [ ]:
data_file = h5py.File(HDF5_PATH, 'r')

In [ ]:
train_seq = HDF5Sequence(data_file, batch_size, 
                         index_path=DATA_PATH + 'train_indices.csv')
val_seq = HDF5Sequence(data_file, batch_size, 
                       index_path=DATA_PATH + 'val_indices.csv')

In [ ]:
len(train_seq)

## Random hyperparam search

In [ ]:
import keras
import keras.layers as KL
import keras.models as KM
from constants import *

ckpt_path_base = './models/checkpoints/bi_rnn_test_{}_{}/'
model = None


while True:
    del model
    
    shape_1 = np.random.randint(16, 257)
    shape_2 = np.random.randint(16, shape_1 + 1)
            
    ckpt_path = ckpt_path_base.format(shape_1, shape_2) + '{epoch:02d}-{val_loss:.4f}.hdf5'

    checkpoint = ModelCheckpoint(ckpt_path, monitor='val_loss', 
                                 verbose=1, save_best_only=False, mode='max')
    logger = keras.callbacks.CSVLogger('./logs/brnn_{}_{}.log'.format(shape_1, shape_2))
    callbacks_list = [checkpoint, logger]
        
    input_layer = KL.Input(shape=(TIMESTEPS, DIM_MELODY), name='input')
    layer = KL.Bidirectional(
        KL.CuDNNLSTM(shape_1, return_sequences=True, name='bi_lstm_1'))(input_layer)
    layer = KL.Bidirectional(KL.CuDNNLSTM(shape_2, name='bi_lstm_2'))(layer)
    output_layer = KL.Dense(DIM_LATENT, activation='linear', name='output')(layer)
    model = KM.Model(inputs=input_layer, outputs=output_layer)

    model.summary()
    
    model.compile(optimizer=keras.optimizers.rmsprop(lr=3e-4),
              loss='mean_squared_error')
    
    model.fit_generator(train_seq, steps_per_epoch=len(train_seq),
                    validation_data=val_seq, validation_steps=len(val_seq),
                    max_queue_size=128, workers=32, epochs=10,
                    callbacks=callbacks_list)

In [ ]:
shape_1

In [ ]:
abc = None
del abc

## Define model: bidirectional RNN

In [ ]:
shape_1 = 128
shape_2 = 128

ckpt_path_base = './models/checkpoints/brnn_{}_{}/'
ckpt_path = ckpt_path_base.format(shape_1, shape_2) + '{epoch:02d}-{val_loss:.4f}.hdf5'

checkpoint = ModelCheckpoint(CHECKPOINT_PATH, monitor='val_loss', verbose=1, save_best_only=False, mode='max')
logger = keras.callbacks.CSVLogger('./logs/brnn_{}_{}.log'.format(shape_1, shape_2))
callbacks_list = [checkpoint, logger]

In [ ]:
import keras
import keras.layers as KL
import keras.models as KM
from constants import *

input_layer = KL.Input(shape=(TIMESTEPS, DIM_MELODY), name='input')
layer = KL.Bidirectional(
    KL.CuDNNLSTM(shape_1, return_sequences=True, name='bi_lstm_1'))(input_layer)
layer = KL.Bidirectional(KL.CuDNNLSTM(shape_2, name='bi_lstm_2'))(layer)
output_layer = KL.Dense(DIM_LATENT, activation='linear', name='output')(layer)
model = KM.Model(inputs=input_layer, outputs=output_layer)

model.summary()

In [ ]:
model.compile(optimizer=keras.optimizers.rmsprop(lr=3e-4),
              loss='mean_squared_error')

In [ ]:
model.fit_generator(train_seq, steps_per_epoch=len(train_seq),
                    validation_data=val_seq, validation_steps=len(val_seq),
                    max_queue_size=128, workers=32, epochs=20,
                    callbacks=callbacks_list)

## Inference tests

In [ ]:
model = keras.models.load_model('./models/checkpoints/brnn_128_128/03-0.1935.hdf5')

In [ ]:
which = 7
batch = val_seq.__getitem__(0)
notes, latents = batch
notes = notes[which]
latents = latents[which]

In [ ]:
# NOTE: in active dev. Use at your own risk.

from copy import deepcopy
from magenta.music.sequences_lib import concatenate_sequences

midi = None
# file = './data/lmd_clean/raw/Michael Jackson/Smooth Criminal.mid'
file = './data/lmd_clean/raw/Simon & Garfunkel/Le Laureat: Mrs. Robinson.mid'
# file = '/home/whillikers/dl/potter.mid'
with open(file, 'rb') as midi_file:
    midi = midi_file.read()
    
ns_full = mm.midi_to_sequence_proto(midi)

# del ns_full.tempos[:]
# del ns_full.time_signatures[:]
# del ns_full.control_changes[:]
# ns_full = utils.strip_to_melody(ns_full)
# ns_melody = deepcopy(ns_full)

# mm.plot_sequence(ns_full)
# mm.play_sequence(ns_full, synth=mm.fluidsynth)

# mel_tensor = config.data_converter._melody_converter.to_tensors(ns_full)
# mel_tensor_2 = mel_tensor.outputs[0]
# mel_tensor_2 = np.pad(mel_tensor_2, [(0, 256 - mel_tensor_2.shape[0]), (0, 0)], 'constant')
# latents_pred = model.predict(np.expand_dims(mel_tensor_2[:256, :], 0))

trio_tensors = config.data_converter.to_tensors(ns_full)[1]
ns_trio = config.data_converter.to_notesequences(trio_tensors)[0]
ns_melody = utils.strip_to_melody(ns_trio)

mm.plot_sequence(ns_trio)
mm.play_sequence(ns_trio, synth=mm.fluidsynth)

melody_tensor = trio_tensors[0][:, :90]
latents_pred = model.predict(np.expand_dims(melody_tensor, 0))[0]
latents_pred = np.expand_dims(latents_pred, 0)

ns_out = concatenate_sequences(
    model_pretrained.decode(latents_pred, temperature=.1, length=64)
)

ns_stitch = utils.remove_melody(ns_out)
ns_stitch.notes.extend(ns_melody.notes)
# ns_stitch = ns_out

mm.plot_sequence(ns_stitch)
mm.play_sequence(ns_stitch, synth=mm.fluidsynth)

In [ ]:
mel_tensor_2.shape

In [ ]:
mel_tensor_2 = mel_tensor.outputs[0]
mel_tensor_2 = np.pad(mel_tensor_2, [(0, 256 - mel_tensor_2.shape[0]), (0, 0)], 'constant')
mel_tensor_2.shape

In [ ]:
latents_pred.shape

In [ ]:
# from magenta.

configs.CONFIG_MAP['hierdec-mel_16bar'].data_converter._to_tensors(ns_full)

In [ ]:
ns_full

In [ ]:
mel_tensor.outputs

In [ ]:
mel_tensor = config.data_converter._melody_converter.to_tensors(ns_full)

In [ ]:
mel_tensor[0][0].shape

In [ ]:
len(config.data_converter.to_tensors(ns_full).outputs)

In [ ]:
len(config.data_converter.to_notesequences(trio_tensors))

In [ ]:
out = config.data_converter.to_notesequences(trio_tensors)

In [ ]:
len(out)

In [ ]:
trio_tensors[0].shape

In [ ]:
model_pretrained = TrainedModel(
    config, batch_size=batch_size,
    checkpoint_dir_or_path='./models/pretrained/{}.ckpt'.format(
        PRETRAINED_MODEL_NAME))

ns = config.data_converter._melody_converter.to_notesequences(np.expand_dims(notes, 0))
mm.plot_sequence(ns[0])
mm.play_sequence(ns[0], synth=mm.fluidsynth)

latents_pred = model.predict(np.expand_dims(notes, 0))[0]

ns_out = model_pretrained.decode([latents, latents_pred],
                                 temperature=0.1, length=64)
ns_true = ns_out[0]
ns_infer = ns_out[1]

ns_stitch = utils.remove_melody(ns_out[1])
ns_stitch.notes.extend(ns[0].notes)

mm.plot_sequence(ns_true)
mm.play_sequence(ns_true, synth=mm.fluidsynth)

mm.plot_sequence(ns_infer)
mm.play_sequence(ns_infer, synth=mm.fluidsynth)

mm.plot_sequence(ns_stitch)
mm.play_sequence(ns_stitch, synth=mm.fluidsynth)